In [1]:
%load_ext autoreload
%autoreload 2


In [2]:

from buttermilk import BM
from promptflow.tracing import trace, start_trace
from rich import print as rprint

import os

from hydra import initialize, compose
from omegaconf import OmegaConf

# Load config, specifying overrides for our particular job
with initialize(version_base=None, config_path="./conf"):
    cfg = compose(config_name='config',
                  overrides=["+data=trans", "+step=trans", "+save=bq", "job=score","source=notebook"])

# Load the main ButterMilk singleton instance
# This takes care of credentials, save paths, and other defaults
bm = BM(cfg=cfg)

# Print config
rprint(OmegaConf.to_container(bm.cfg, resolve=True))



/opt/homebrew/Caskroom/miniconda/base/envs/bm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2024-11-06 12:31:08 J5HW6L4KT6 buttermilk buttermilk.py[ 297] INFO {'message': "Logging setup for: 20241106T0231Z-AviX-J5HW6L4KT6-suzor. Ready for data collection, saving log to Google Cloud Logs (Resource(type='generic_task', labels={'project_id': 'dmrc-platforms', 'location': 'us-central1', 'namespace': 'automod', 'job': 'score', 'task_id': '20241106T0231Z-AviX-J5HW6L4KT6-suzor'})). Default save directory for data in this run is: gs://dmrc-analysis/runs/automod/score/20241106T0231Z-AviX-J5HW6L4KT6-suzor", 'project': 'automod', 'job': 'score', 'run_id': '20241106T0231Z-AviX-J5HW6L4KT6-suzor', 'save_dir': 'gs://dmrc-analysis/runs/automod/score/20241106T0231Z-AviX-J5HW6L4KT6-suzor', 'ip': '159.196.210.27', 'node_name': 'J5HW6L4KT6', 'username': 'suzor'}
2024-11-06 12:31:08 J5HW6L4KT6 buttermilk buttermilk.py[ 305] DEBUG Buttermilk version is: 0.2.0
Prompt flow service has started...


{
    'name': 'automod',
    'job': 'score',
    'source': 'notebook',
    'secret_provider': 'azure',
    'logger': 'gcp',
    'tracing': True,
    'verbose': True,
    'models_secret': 'models',
    'save_dest': 'gcp',
    'save_dir': None,
    'gcp': {'project': 'dmrc-analysis', 'region': 'us-central1', 'bucket': 'dmrc-analysis'},
    'azure': {'vault': 'https://suzorvault.vault.azure.net/', 'resource_group': 'rg-suzor_ai'},
    'aws': {'secret_name': 'secret', 'region': 'us-east-1'},
    'step': [
        {
            'flow': 'judger',
            'num_runs': 1,
            'concurrent': 20,
            'agent': {
                'name': 'lc',
                'template': 'judge',
                'criteria': [
                    'trans_cte',
                    'trans_tja',
                    'trans_glaad',
                    'trans_hrc',
                    'trans_simplified',
                    'trans_factored'
                ],
                'formatting': 'json_rules',
                'model': ['haiku', 'llama31_70b', 'gpt4o', 'sonnet', 'gemini15pro']
            },
            'parameters': {'record_id': 'record_id', 'content': 'content', 'title': 'title'},
            'data': None
        },
        {
            'flow': 'synth',
            'num_runs': 1,
            'concurrent': 20,
            'agent': {
                'name': 'lc_judge',
                'template': 'synthesise',
                'criteria': ['trans_simplified'],
                'formatting': 'json_rules',
                'model': ['haiku', 'gpt4o', 'sonnet', 'gemini15pro']
            },
            'parameters': {'answers': 'answers', 'meta': 'meta'},
            'data': {
                'judger': {
                    'name': 'previous runs',
                    'type': 'job',
                    'dataset': 'dmrc-analysis.toxicity.flow',
                    'max_records_per_group': 1,
                    'filter': {'agent_info.flow': 'judger', 'outputs.reasons': ['NOTNULL']},
                    'group': {
                        'record_id': 'record.record_id',
                        'flow': 'agent_info.flow',
                        'template': 'agent_info.template',
                        'criteria': 'agent_info.template_vars.criteria',
                        'model': 'parameters.model'
                    },
                    'columns': {
                        'answers': {
                            'id': 'job_id',
                            'model': 'model',
                            'criteria': 'criteria',
                            'template': 'template',
                            'flow': 'flow',
                            'reasons': 'outputs.reasons'
                        },
                        'meta': {
                            'flow': 'agent_info.flow',
                            'template': 'agent_info.template',
                            'criteria': 'agent_info.template_vars.criteria',
                            'model': 'parameters.model',
                            'prediction': 'outputs.prediction',
                            'labels': 'outputs.labels',
                            'confidence': 'outputs.confidence',
                            'timestamp': 'timestamp'
                        }
                    }
                }
            }
        },
        {
            'flow': 'eval',
            'num_runs': 1,
            'concurrent': 20,
            'agent': {
                'name': 'lc',
                'template': 'differences',
                'criteria': ['trans_simplified'],
                'model': ['haiku', 'gpt4o', 'sonnet', 'gemini15pro']
            },
            'parameters': {'answers': 'answers', 'meta': 'meta'},
            'data': {
                'judger': {
                    'name': 'previous runs and synth',
                    'type': 'job',
                    'dataset': 'dmrc-analysis.toxicity.flow',
                    'max_records_per_group': 1,
      

## First step assesses content against our criteria

In [4]:
# Create an orchestrator to conduct all combinations of jobs we want to run
from buttermilk.runner.orchestrator import MultiFlowOrchestrator
orchestrator = MultiFlowOrchestrator(step=cfg.step[0], data=cfg.data, save=cfg.save, source=cfg.source)


In [4]:
await orchestrator.run_tasks()


2024-11-06 11:15:48 J5HW6L4KT6 buttermilk orchestrator.py[  81] DEBUG Starting task for Agent lc_Y63Jb8 with job cLWruB8mzRSZNGYQ7mHPPV.


DEBUG:buttermilk:Starting task for Agent lc_Y63Jb8 with job cLWruB8mzRSZNGYQ7mHPPV.


2024-11-06 11:17:00 J5HW6L4KT6 buttermilk lc.py[ 204] DEBUG Invoking chain with haiku...


DEBUG:buttermilk:Invoking chain with haiku...


2024-11-06 11:17:01 J5HW6L4KT6 buttermilk orchestrator.py[  81] DEBUG Starting task for Agent lc_Y63Jb8 with job XSG8q7NYvpXe6Egjd6J538.


...

DEBUG:buttermilk:Agent lc_JsEd9i completed job 7AdXQXHinfh39372DwREGu successfully.


2024-11-06 11:31:40 J5HW6L4KT6 buttermilk orchestrator.py[ 106] INFO Completed run 1 of 1


INFO:buttermilk:Completed run 1 of 1


2024-11-06 11:31:40 J5HW6L4KT6 buttermilk orchestrator.py[ 110] INFO All tasks have completed.


INFO:buttermilk:All tasks have completed.


## Second step is a synthesis of previous draft answers

In [5]:
# Create an orchestrator to conduct all combinations of jobs we want to run
from buttermilk.runner.orchestrator import MultiFlowOrchestrator
orchestrator = MultiFlowOrchestrator(step=cfg.step[1], data=cfg.data, save=cfg.save, source=cfg.source)


2024-11-06 12:33:39 J5HW6L4KT6 buttermilk buttermilk.py[ 376] INFO Query stats: Ran in 0:00:06.161060 seconds, cache hit: False, billed 15.73 MB, approx cost $7.9e-06.


INFO:buttermilk:Query stats: Ran in 0:00:06.161060 seconds, cache hit: False, billed 15.73 MB, approx cost $7.9e-06.


<class 'pandas.core.frame.DataFrame'>
MultiIndex: 350 entries, ('transgender_paralympian_dailymail', 'judger', 'judge', 'trans_tja', 'llama31_70b') to ('transgender_paralympian_BBC', 'judger', 'judge', 'trans_hrc', 'gpt4o')
Data columns (total 22 columns):
 #   Column      Non-Null Count  Dtype              
---  ------      --------------  -----              
 0   id          350 non-null    object             
 1   timestamp   350 non-null    datetime64[us, UTC]
 2   run_info    350 non-null    object             
 3   agent_info  350 non-null    object             
 4   source      350 non-null    object             
 5   record      350 non-null    object             
 6   parameters  350 non-null    object             
 7   outputs     350 non-null    object             
 8   error       0 non-null      object             
 9   metadata    350 non-null    object             
 10  prompt      350 non-null    object             
 11  record_id   350 non-null    object             
 

In [6]:
# Inspect the dataset so we can make sure the inputs look ok
rprint(orchestrator._dataset.sample().answers.values)


[list([{'id': 'J6SqUtdwyQX5uuLk5hWCYa', 'model': 'gemini15pro', 'criteria': 'trans_cte', 'template': 'judge', 
'flow': 'judger', 'reasons': ["The article discusses the murder of Tai'Vion Lathan, a Black transgender woman, 
highlighting the increased violence faced by the transgender community, particularly transgender women of color.  
It uses Lathan's correct name and pronouns throughout the piece. The article aims to raise awareness about this 
issue and advocate for justice and policy change to protect transgender individuals. The article's tone is 
respectful and aims to humanize Lathan, emphasizing her personality, family, and aspirations.", "The content 
consistently uses Tai'Vion Lathan's chosen name and pronouns (she/her) throughout the article, as requested by her 
family. This aligns with the provided guidelines from the Associated Press Stylebook and the New York Times Style 
Book.", 'The article specifically discusses the violence faced by transgender people, especially transgender women 
of color. It highlights the statistics on transgender homicides and discusses the broader context of legislative 
attacks on transgender rights and healthcare. The language used is respectful and aims to raise awareness of the 
issue.', "The article focuses on the tragic murder of a transgender woman and the broader issues faced by the 
transgender community. By highlighting Lathan's personal life, aspirations, and the grief of her family, the 
article humanizes her and counters the dehumanization often experienced by transgender individuals.", 'The article 
promotes respect and understanding of transgender people by using accurate terminology and highlighting the 
importance of recognizing their full humanity, beyond their gender identity. It also advocates for justice and 
policy change to protect transgender individuals from violence and discrimination.']}, {'id': 
'Qgi5p3fnPShknhHsTNBf2X', 'model': 'gpt4o', 'criteria': 'trans_cte', 'template': 'judge', 'flow': 'judger', 
'reasons': ['The content is a news report covering the death of Tai’Vion Lathan, a transgender woman, who was 
murdered in Baltimore. It discusses the circumstances of her death, the grief of her family, and provides 
statistics on violence against transgender individuals, especially Black transgender women.', "According to the 
guidelines, the pronouns and names used should be those preferred by the individual. The content uses 'Tai’Vion 
Lathan,' also known as 'Tai' and uses 'her' and 'she' pronouns throughout, indicating adherence to this guideline 
as there are no signs to contrary regarding Lathan's preferred pronouns.", "The content does cite Tai’Vion Lathan's
transgender status, but it is pertinent to the context, as the article addresses violence against transgender 
individuals and highlights ongoing societal issues faced by this community. Therefore, this mention is justified as
it provides context that is necessary for understanding the motive of the crime and its broader social 
implications.", "According to the guidelines, terminology related to transgender individuals should be used 
sensitively and accurately. Terms like 'transgender woman' are used in the article, which aligns correctly with 
guideline terminology.", "The article provides national context concerning issues faced by transgender people, such
as violence and anti-trans legislation, which align with key issues identified within the guidelines. It 
sensitively incorporates this information to elevate public awareness of the broader context in which Tai’Vion 
Lathan's murder occurred."]}, {'id': 'h2yUoJ6ii2YUwdJ5cgvF8z', 'model': 'haiku', 'criteria': 'trans_cte', 
'template': 'judge', 'flow': 'judger', 'reasons': ["The content provided is a news article reporting on the tragic 
murder of a 24-year-old transgender woman named Tai'Vion Lathan in Baltimore, Maryland. The article provides 
details about Lathan's life, the circumstances of her death, and the broader context of violence fa

In [8]:
await orchestrator.run_tasks()


2024-11-06 12:38:44 J5HW6L4KT6 buttermilk orchestrator.py[  81] DEBUG Starting task for Agent lc_judge_YNpFkj with job ZoRBmAyKKxiPZhU4CyYZ76.


DEBUG:buttermilk:Starting task for Agent lc_judge_YNpFkj with job ZoRBmAyKKxiPZhU4CyYZ76.


2024-11-06 12:38:44 J5HW6L4KT6 buttermilk templating.py[  50] DEBUG Unable to decode template as Prompty: Illegal formatting of prompty. The prompt file is in markdown format and can be divided into two parts, the first part is in YAML format and contains connection and model information. The second part is the prompt template., e.args=('Illegal formatting of prompty. The prompt file is in markdown format and can be divided into two parts, the first part is in YAML format and contains connection and model information. The second part is the prompt template.',)


DEBUG:buttermilk:Unable to decode template as Prompty: Illegal formatting of prompty. The prompt file is in markdown format and can be divided into two parts, the first part is in YAML format and contains connection and model information. The second part is the prompt template., e.args=('Illegal formatting of prompty. The prompt file is in markdown format and can be divided into two parts, the first part is in YAML format and contains connection and model information. The second part is the prompt template.',)


2024-11-06 12:38:44 J5HW6L4KT6 buttermilk lc.py[ 204] DEBUG Invoking chain with haiku...


...

DEBUG:buttermilk:Agent lc_judge_YNpFkj completed job ZoRBmAyKKxiPZhU4CyYZ76 successfully.


2024-11-06 12:49:15 J5HW6L4KT6 buttermilk orchestrator.py[ 106] INFO Completed run 1 of 1


INFO:buttermilk:Completed run 1 of 1


2024-11-06 12:49:15 J5HW6L4KT6 buttermilk orchestrator.py[ 110] INFO All tasks have completed.


INFO:buttermilk:All tasks have completed.


## Third step, we summarise the answers, similarities, and differences

In [9]:
# Create an orchestrator to conduct all combinations of jobs we want to run
from buttermilk.runner.orchestrator import MultiFlowOrchestrator
orchestrator = MultiFlowOrchestrator(step=cfg.step[2], data=cfg.data, save=cfg.save, source=cfg.source)


2024-11-06 15:08:06 J5HW6L4KT6 buttermilk buttermilk.py[ 376] INFO Query stats: Ran in 0:00:03.769667 seconds, cache hit: False, billed 22.02 MB, approx cost $1.1e-05.


INFO:buttermilk:Query stats: Ran in 0:00:03.769667 seconds, cache hit: False, billed 22.02 MB, approx cost $1.1e-05.


In [10]:
await orchestrator.run_tasks()


2024-11-06 15:09:03 J5HW6L4KT6 buttermilk orchestrator.py[  81] DEBUG Starting task for Agent lc_H2Cy74 with job ZCss8xW3KHbnFdkAJ3hUoN.


DEBUG:buttermilk:Starting task for Agent lc_H2Cy74 with job ZCss8xW3KHbnFdkAJ3hUoN.


2024-11-06 15:09:03 J5HW6L4KT6 buttermilk lc.py[ 204] DEBUG Invoking chain with haiku...


DEBUG:buttermilk:Invoking chain with haiku...


2024-11-06 15:09:04 J5HW6L4KT6 buttermilk orchestrator.py[  81] DEBUG Starting task for Agent lc_H2Cy74 with job dLzUUufZDsER7RGHeYAXb7.


...

DEBUG:buttermilk:Agent lc_h5QtST completed job HSdm245NueuhHwmYTA9iCC successfully.


2024-11-06 15:11:21 J5HW6L4KT6 buttermilk orchestrator.py[ 106] INFO Completed run 1 of 1


INFO:buttermilk:Completed run 1 of 1


2024-11-06 15:11:21 J5HW6L4KT6 buttermilk orchestrator.py[ 110] INFO All tasks have completed.


INFO:buttermilk:All tasks have completed.
